In [ ]:
import numpy as np
import pandas as pd
from sqlalchemy import text
from connect_postgres import connect_to_db

from keras.models import Sequential
from keras.layers import LSTM, Dense
from data_loader import load_data_from_db

## Loading Postgres Data

In [ ]:
def load_data_from_db(window_size):
    conn = connect_to_db()
    df = pd.read_sql(
        text("SELECT timestamp, flow_rate, usage_last_hour FROM water_readings ORDER BY timestamp"),
        conn,
        parse_dates=['timestamp']
    )
    conn.close()
    values = df[['flow_rate','usage_last_hour']].to_numpy()
    X = []
    y = []
    for i in range(len(values) - window_size):
        X.append(values[i:i + window_size])
        y.append(values[i + window_size, 1])
    return np.array(X), np.array(y)

## Estimating Future Water Usage

In [ ]:
window_size = 24
X, y = load_data_from_db(window_size)

model = Sequential([
    LSTM(64, input_shape=(window_size,2), return_sequences=True),
    LSTM(32),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)
model.save('../models/water_model.h5')
